In [2]:
import pandas as pd

df = pd.read_excel('../data/cities_streetview_run_1.xlsx', engine='openpyxl')

# Filter the rows based on the date condition
print("CITIES STREETVIEWED: ")
print("within the last year: ",len(df[df['date'] >= '2022-11']))
print("within the last 6 months: ",len(df[df['date'] >= '2023-05']))
print("within the past 3 months: ",len(df[df['date'] >= '2023-08']))
print("within the past month: ",len(df[df['date'] >= '2023-10']))

CITIES STREETVIEWED: 
within the last year:  273
within the last 6 months:  137
within the past 3 months:  45
within the past month:  3


In [ ]:
import requests
from utils import request_streetview

## GRAB SINGLE IMAGE, AND PASS IT TO THE MODEL
location='48.486132,-123.325421'
params = {
    "location": location,
    "size": '640x300',
    "return_error_code": 'true',
    "fov": 120,
    "heading": 0,
    "pitch": 0,
    "source": "outdoor",
}
# pano_id are dynamic, use location instead

for heading, direction in zip([0, 90, 180, 270], ['N', 'E', 'S', 'W']):
    params['heading'] = heading
    request_streetview(params, f"../images/{location}_{direction}.jpg")
